# Saint Joseph City Government Text Analysis

## Why do this?

I'm going about this for a few reasons:

1) Build some skill in data analysis and parsing  
2) Satisfy some curiosities I have with how local government operates  
3) Eventually use this set to examine local governments in a number of areas  
4) help inform me about the policies of elected officials  
5) have an interesting set of potential factors when comparing to other indicators, like economic response  

## 1. Gather data from SJCity database

First, using the excellent [Web Scraper](https://chrome.google.com/webstore/detail/web-scraper/jnhgnonknehpejjnehehllkliplmbmhn?utm_source=chrome-ntp-icon) tool, I scrape the [Saint Joseph City Database](http://stjosephcitymi.iqm2.com/Citizens/calendar.aspx?From=1%2f1%2f1900&To=12%2f31%2f9999) for relevant text and pdf links for meeting minutes.

Now, with the set of links, I can start automatically downloading the PDFs of the agenda, the agenda packet, the summary, and the meeting minutes for every meeting since 2013.

In [173]:
import pandas as pd

linkData = pd.read_csv("saint_joseph_city_meeting_calendar_full.csv")

print(linkData[0:5])

  web-scraper-order                              web-scraper-start-url  \
0    1558406890-637  http://stjosephcitymi.iqm2.com/Citizens/calend...   
1    1558406890-472  http://stjosephcitymi.iqm2.com/Citizens/calend...   
2    1558406890-816  http://stjosephcitymi.iqm2.com/Citizens/calend...   
3    1558406890-526  http://stjosephcitymi.iqm2.com/Citizens/calend...   
4    1558406890-880  http://stjosephcitymi.iqm2.com/Citizens/calend...   

               datetime                                       meeting-type  \
0  Nov 15, 2017 4:00 PM     Water Services Joint Operating Board - Regular   
1  Jun 15, 2016 4:00 PM     Water Services Joint Operating Board - Regular   
2  Mar 11, 2019 6:00 PM                  City Commission - Regular Meeting   
3   Dec 7, 2016 4:00 PM  Brownfield Redevelopment Authority - Regular M...   
4  Aug 29, 2019 9:00 AM                           Cemetery Board - Regular   

   agenda                                        agenda-href  agenda-packet  \
0  Agen

Looking at the data, there are quite a few NaNs.  In most cases, it looks like these are meetings that were cancelled.  Normally, I'd just clean those rows.  However, viewing the number of meetings cancelled over time could be interesting.  I'll make a copy of the dataframe to start cleaning a different version.


In [174]:
ld = linkData.copy().dropna(axis=0, how='any')
ld.datetime = ld.datetime.replace({':':'_'},regex=True)

print(ld[ld.datetime.str.len() == 12])

    web-scraper-order                              web-scraper-start-url  \
8      1558406890-856  http://stjosephcitymi.iqm2.com/Citizens/calend...   
39     1558406890-729  http://stjosephcitymi.iqm2.com/Citizens/calend...   
57     1558406890-726  http://stjosephcitymi.iqm2.com/Citizens/calend...   
98     1558406890-740  http://stjosephcitymi.iqm2.com/Citizens/calend...   
179    1558406890-748  http://stjosephcitymi.iqm2.com/Citizens/calend...   
229    1558406890-544  http://stjosephcitymi.iqm2.com/Citizens/calend...   
259    1558406890-879  http://stjosephcitymi.iqm2.com/Citizens/calend...   
263    1558406890-771  http://stjosephcitymi.iqm2.com/Citizens/calend...   
294    1558406890-867  http://stjosephcitymi.iqm2.com/Citizens/calend...   
300    1558406890-655  http://stjosephcitymi.iqm2.com/Citizens/calend...   
356    1558406890-703  http://stjosephcitymi.iqm2.com/Citizens/calend...   
380    1558406890-877  http://stjosephcitymi.iqm2.com/Citizens/calend...   
405    15584

Some meetings don't have recorded times. I'll have to fix these to get consistent datetime objects.

In [175]:
ld['datetime'] = ld.datetime.apply(lambda s: s + ' 12_00 AM' if len(s) == 12 else s)

Also, it might be useful to create a smaller dataframe for testing.

In [176]:
ld_small = ld[0:5]
for i,j in ld_small.iterrows():
    print(ld["agenda-href"][i])

http://stjosephcitymi.iqm2.com/Citizens/FileOpen.aspx?Type=14&ID=1631&Inline=True
http://stjosephcitymi.iqm2.com/Citizens/FileOpen.aspx?Type=14&ID=1784&Inline=True
http://stjosephcitymi.iqm2.com/Citizens/calendar.aspx?From=1%2f1%2f1900&To=12%2f31%2f9999
http://stjosephcitymi.iqm2.com/Citizens/FileOpen.aspx?Type=14&ID=1573&Inline=True
http://stjosephcitymi.iqm2.com/Citizens/FileOpen.aspx?Type=14&ID=1150&Inline=True


In [177]:
import datetime 
print(ld['datetime'])
ld['dto'] = pd.to_datetime(ld['datetime'], format="%b %d, %Y %I_%M %p") 
#for i,j in ld.iteritems():
    #ld[i]["datetime"] = datetime.strptime(ld[i])
#    if i == "datetime":
#        dto.append(datetime.strptime(ld[i][j], "%b %d, %Y %I%M %p"))
#        print(dto)
    #ld.insert(3, "dto", dto, True)

0       Nov 15, 2017 4_00 PM
2       Mar 11, 2019 6_00 PM
4       Aug 29, 2019 9_00 AM
5      Jun 28, 2017 12_00 PM
6      Oct 22, 2014 12_00 PM
7        Mar 5, 2015 4_30 PM
8      Jun 26, 2019 12_00 AM
9       Jun 20, 2018 4_00 PM
10      Feb 18, 2016 8_30 AM
11      Aug 19, 2015 4_00 PM
13      Oct 17, 2016 6_00 PM
14      May 27, 2015 8_00 AM
15     Jul 30, 2014 12_00 PM
17      Apr 25, 2019 9_00 AM
18      Apr 27, 2017 9_00 AM
19      Mar 28, 2016 6_00 PM
20       Dec 8, 2014 6_00 PM
21      Nov 11, 2013 7_45 PM
22      Jun 22, 2017 5_00 PM
23     Apr 30, 2014 12_00 PM
24      Jan 17, 2018 4_00 PM
26      Jun 22, 2015 6_00 PM
27      Dec 19, 2019 4_30 PM
28      May 3, 2017 11_30 AM
29       Feb 6, 2019 5_30 PM
30      Nov 10, 2014 6_00 PM
31       Jan 7, 2016 4_30 PM
32       Apr 9, 2019 9_00 AM
33       Dec 1, 2016 4_30 PM
34      Oct 15, 2015 8_30 AM
               ...          
664     Jul 28, 2014 6_00 PM
666     Oct 23, 2017 7_00 PM
667      May 6, 2019 6_00 PM
668     Feb 19

In [180]:
import requests
from time import sleep

s = "C:\\Users\\alexh\\Documents\\Projects\\SJCommission\\pdfs\\"
for index, row in ld.iterrows():
    agString = s + ld["datetime"][index] + " - " +ld["meeting-type"][index] + " - agenda.pdf"
    apString = s + ld["datetime"][index] + " - " +ld["meeting-type"][index] + " - agenda-packet.pdf"
    suString = s + ld["datetime"][index] + " - " +ld["meeting-type"][index] + " - summary.pdf"
    miString = s + ld["datetime"][index] + " - " +ld["meeting-type"][index] + " - minutes.pdf"
    
    #don't try to collect future documents
    if ld.loc[index, 'dto'] < datetime.datetime.now():
        r = requests.get(ld["agenda-href"][i])
        with open(agString, 'wb') as fd:
            for chunk in r.iter_content(chunk_size=128):
                fd.write(chunk)
        sleep(2)
        r = requests.get(ld["agenda-packet-href"][i])
        with open(apString, 'wb') as fd:
            for chunk in r.iter_content(chunk_size=128):
                fd.write(chunk)
        sleep(2)
        r = requests.get(ld["summary-href"][i])
        with open(suString, 'wb') as fd:
            for chunk in r.iter_content(chunk_size=128):
                fd.write(chunk)
        sleep(2)
        r = requests.get(ld["minutes-href"][i])
        with open(miString, 'wb') as fd:
            for chunk in r.iter_content(chunk_size=128):
                fd.write(chunk)
        sleep(2)

After Reviewing the files downloaded in the first small batch, it seems as though the script isn't handling the NaNs properly.  The Aug 29, 2019 Cemetery Board files aren't being read properly, as they haven't been made yet.  I would've thought that those rows would have been omitted. 

To solve this, and not make unnecessary requests, I'm going to make datetime objects from the meeting strings, and only download ones that have already happened.


### Getting text from the PDF files

Now that I've got all the files I need, it's time to make them more useful.  After using a few different approaches in the past, like pytesseract,  I'm fortunate that these PDF files seem to play well with PyPDF. Time to traverse all 2,102, saving the text data!

In [191]:
import PyPDF2
from os import listdir
from os.path import isfile, join

path = 'C:\\Users\\alexh\\Documents\\Projects\\SJCommission\\pdfs'
#save the filename, to enable easier concatenation to the dataframe
files = [[f,join(path,f)] for f in listdir(path) if isfile(join(path, f))] 

In [ ]:
def read_text(filePath):
    doc_content = ""
    with open(testPath, 'rb') as pdf_file:
        read_pdf = PyPDF2.PdfFileReader(pdf_file)
        num_pages = read_pdf.getNumPages()
        for i in range(num_pages):
            page = read_pdf.getPage(i)
            doc_content = doc_content + page.extractText()
    return doc_content
docdata = []  #[filename, contents]
for fileGroup in files:
    docdata.append([fileGroup[0],read_text(fileGroup[1])])
    